# Deploy TensorFlow model to ACI


In the previous lab, we used `Hyperdrive` to tune FCNN top model. The model was registered in Model Registry.

Now, we are ready to deploy the model.

The model can be deployed to a variety of target runtimes, including:
- Azure Container Instance
- Azure Kubernetes Service
- IoT Edge
- FPGA


In this lab, we will deploy the model as a web service in Azure Container Instance.

![AML Arch](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/amlarch.png)

## Connect to the workspace

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.2


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()

Found the config file in: /data/home/demouser/notebooks/MTC_AzureAILabs/ML-DeepLearningOnAzure/aml_config/config.json


## Deploy as web service

To build the correct environment for ACI, provide the following:
* A scoring script that invokes the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before


### Create scoring script

Create the scoring script, called score.py, used by the web service call to invoke the model.

You must include two required functions in the scoring script:
* The `init()` function, which loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats can be used.

In [3]:
%%writefile score.py
import json
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import load_model

from azureml.core.model import Model

import azureml.contrib.brainwave.models.utils as utils
from azureml.contrib.brainwave.models import QuantizedResnet50

def init():
    # Instantiate ResNet50 featurizer
    global featurizer
    
    # Create ResNet50 
    model_path = os.path.expanduser('~/models')
    featurizer = QuantizedResnet50(model_path, is_frozen=True)

    # Load the pretrained top
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path(model_name = '<<modelid>>')
    #model = load_model(model_path)
    print(model_path)
  

def run(raw_data):
    try:
        # convert json to numpy array
        images = np.array(json.loads(raw_data)['data'])
        # normalize as required by ResNet50
        
        predictions = str(images.shape)
        
        # Call the top
        # predictions = model.predict(features)
        # Add string labels
        #labels = ["Barren",
        #          "Cultivated",
        #          "Developed",
        #          "Forest",
        #          "Herbaceous",
        #          "Shrub"]
        
        # Get string labels for predictions
        #string_predictions = [labels[pred] for pred in predictions]
        
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    
    # Return both numeric and string predictions
    # return json.dumps({"predictions": predictions.tolist(), "labels": string_predictions})
    return json.dumps({"predictions": predictions})

Overwriting score.py


Substitute the actual model ID in the script file.

In [4]:
from azureml.core.model import Model

model_name = 'aerial-classifier-brainwave'
model = Model(ws, name=model_name)
script_file_name = 'score.py'

with open(script_file_name, 'r') as cefr:
    content = cefr.read()
    
with open(script_file_name, 'w') as cefw:
    cefw.write(content.replace('<<modelid>>', model.name))

In [5]:
model.get_model_path(model_name)

ModelNotFoundException: Model not found in cache or in root at ./aerial-classifier-brainwave. For more info,set logging level to DEBUG.

Review the updated script.

In [ ]:
with open("score.py","r") as f:
    print(f.read())

### Create a Conda dependencies environment file.

Next, create an environment file that specifies the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. 




In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

mycondaenv = CondaDependencies.create(pip_packages=[
                'h5py', 
                'pillow', 
                'tqdm', 
                'azureml-sdk[contrib]', 
                'scikit-learn', 
                'tensorflow==1.10'])

with open("mydeployenv.yml","w") as f:
    f.write(mycondaenv.serialize_to_string())

Review the content of 'yml' file.

In [ ]:
with open("mydeployenv.yml","r") as f:
    print(f.read())

### Create docker image for deployment

To create a Container Image, you need four things: the model metadata (as retrieved from Model Registry), the scoring script file, the runtime configuration (defining whether Python or PySpark should be used) and the Conda Dependencies file.

In [ ]:
model.get_model_path(model_name)

In [ ]:
from azureml.core.image import ContainerImage, Image

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="mydeployenv.yml",
                                                  description="Image for aerial classifier",
                                                  tags={"Classifier": "FCNN"})

image = Image.create(name = "aerial-classifier-image",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)

### Define ACI configuration

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. The default is 1 core and 1 gigabyte of RAM. Since we are using ResNet50 featurizer we are CPU bound.  In this lab we will use the defaults but you should always go through the proper performance plannig exercise to find the right configuration.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "aerial",  "method" : "classifier"}, 
                                               description='Predict aerial images')

### Deploy in ACI

Deploy the image as a web service in Azure Container Instance.

In [ ]:
from azureml.core.webservice import Webservice

aci_service_name = 'aerial-classifier-svc'
print(aci_service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)

To troubleshoot any deployment issues you can retrieve deployment logs.

In [ ]:
print(aci_service.get_logs())

Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [ ]:
print(aci_service.scoring_uri)

## Test deployed service

You will now test the deployed model with 3 test images.  
 
First, download sample images.

In [ ]:
%%sh
mkdir samples
cd samples
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/barren-1.png
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/cultivated-1.png
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/developed-1.png
ls -l


Define utility function that wraps loading images and invoking the service. The function takes as an input a list of pathnames to images. It loads and converts the binary images into a JSON array and invokes the service passing the JSON payload.

In [ ]:
from PIL import Image
import numpy as np
import json

def score(pathnames, service):
    images = []
    for pathname in pathnames:
        img = Image.open(pathname)
        img = np.asarray(img).tolist()
        images.append(img)
    images = json.dumps({"data": images})
    images = bytes(images, encoding='utf8')
    results = json.loads(service.run(input_data=images))
    return results

Call the service.

In [ ]:
aci_service = Webservice(workspace=ws, name='aerial-automl-svc')

pathnames = ['samples/barren-1.png', 'samples/developed-1.png', 'samples/cultivated-1.png']

results = score(pathnames, aci_service)

print(results)

## Clean up resources

Delete the ACI service.

In [ ]:
aci_service.delete()